# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 24 2022 1:18PM,249265,20,8687945,"Exact-Rx, Inc.",Released
1,Oct 24 2022 1:18PM,249265,20,8687946,"Exact-Rx, Inc.",Released
2,Oct 24 2022 1:18PM,249265,20,8687947,"Exact-Rx, Inc.",Released
3,Oct 24 2022 1:18PM,249265,20,8687948,"Exact-Rx, Inc.",Released
4,Oct 24 2022 1:18PM,249265,20,8687949,"Exact-Rx, Inc.",Released
5,Oct 24 2022 1:18PM,249265,20,8687950,"Exact-Rx, Inc.",Released
6,Oct 24 2022 1:18PM,249265,20,8687951,"Exact-Rx, Inc.",Released
7,Oct 24 2022 1:18PM,249265,20,8687952,"Exact-Rx, Inc.",Released
8,Oct 24 2022 1:18PM,249265,20,8687953,"Exact-Rx, Inc.",Released
9,Oct 24 2022 1:18PM,249265,20,8687954,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,249261,Released,6
18,249262,Released,12
19,249263,Released,6
20,249264,Released,1
21,249265,Released,21


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249261,NaN,NaN,6.0
249262,NaN,NaN,12.0
249263,NaN,NaN,6.0
249264,NaN,NaN,1.0
249265,NaN,NaN,21.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249218,0.0,0.0,1.0
249223,2.0,4.0,1.0
249228,0.0,0.0,60.0
249229,0.0,0.0,59.0
249232,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249218,0,0,1
249223,2,4,1
249228,0,0,60
249229,0,0,59
249232,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249218,0,0,1
1,249223,2,4,1
2,249228,0,0,60
3,249229,0,0,59
4,249232,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249218,,,1
1,249223,2,4,1
2,249228,,,60
3,249229,,,59
4,249232,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 24 2022 1:18PM,249265,20,"Exact-Rx, Inc."
21,Oct 24 2022 12:58PM,249264,10,Yusen – 3D Matrix
22,Oct 24 2022 12:53PM,249262,10,"Methapharm, Inc."
34,Oct 24 2022 12:51PM,249263,10,"Methapharm, Inc."
35,Oct 24 2022 12:51PM,249263,10,Methapharm-G
40,Oct 24 2022 12:47PM,249261,10,"Snap Medical Industries, LLC"
46,Oct 24 2022 12:46PM,249260,10,Yusen – 3D Matrix
47,Oct 24 2022 12:34PM,249259,19,ACG North America LLC
48,Oct 24 2022 12:14PM,249256,19,"AdvaGen Pharma, Ltd"
49,Oct 24 2022 12:06PM,249253,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 24 2022 1:18PM,249265,20,"Exact-Rx, Inc.",,,21
1,Oct 24 2022 12:58PM,249264,10,Yusen – 3D Matrix,,,1
2,Oct 24 2022 12:53PM,249262,10,"Methapharm, Inc.",,,12
3,Oct 24 2022 12:51PM,249263,10,"Methapharm, Inc.",,,6
4,Oct 24 2022 12:51PM,249263,10,Methapharm-G,,,6
5,Oct 24 2022 12:47PM,249261,10,"Snap Medical Industries, LLC",,,6
6,Oct 24 2022 12:46PM,249260,10,Yusen – 3D Matrix,,,1
7,Oct 24 2022 12:34PM,249259,19,ACG North America LLC,,,1
8,Oct 24 2022 12:14PM,249256,19,"AdvaGen Pharma, Ltd",,,1
9,Oct 24 2022 12:06PM,249253,19,"AdvaGen Pharma, Ltd",,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 1:18PM,249265,20,"Exact-Rx, Inc.",21,,
1,Oct 24 2022 12:58PM,249264,10,Yusen – 3D Matrix,1,,
2,Oct 24 2022 12:53PM,249262,10,"Methapharm, Inc.",12,,
3,Oct 24 2022 12:51PM,249263,10,"Methapharm, Inc.",6,,
4,Oct 24 2022 12:51PM,249263,10,Methapharm-G,6,,
5,Oct 24 2022 12:47PM,249261,10,"Snap Medical Industries, LLC",6,,
6,Oct 24 2022 12:46PM,249260,10,Yusen – 3D Matrix,1,,
7,Oct 24 2022 12:34PM,249259,19,ACG North America LLC,1,,
8,Oct 24 2022 12:14PM,249256,19,"AdvaGen Pharma, Ltd",1,,
9,Oct 24 2022 12:06PM,249253,19,"AdvaGen Pharma, Ltd",11,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 1:18PM,249265,20,"Exact-Rx, Inc.",21,,
1,Oct 24 2022 12:58PM,249264,10,Yusen – 3D Matrix,1,,
2,Oct 24 2022 12:53PM,249262,10,"Methapharm, Inc.",12,,
3,Oct 24 2022 12:51PM,249263,10,"Methapharm, Inc.",6,,
4,Oct 24 2022 12:51PM,249263,10,Methapharm-G,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 1:18PM,249265,20,"Exact-Rx, Inc.",21,NaN,NaN
1,Oct 24 2022 12:58PM,249264,10,Yusen – 3D Matrix,1,NaN,NaN
2,Oct 24 2022 12:53PM,249262,10,"Methapharm, Inc.",12,NaN,NaN
3,Oct 24 2022 12:51PM,249263,10,"Methapharm, Inc.",6,NaN,NaN
4,Oct 24 2022 12:51PM,249263,10,Methapharm-G,6,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 1:18PM,249265,20,"Exact-Rx, Inc.",21,0.0,0.0
1,Oct 24 2022 12:58PM,249264,10,Yusen – 3D Matrix,1,0.0,0.0
2,Oct 24 2022 12:53PM,249262,10,"Methapharm, Inc.",12,0.0,0.0
3,Oct 24 2022 12:51PM,249263,10,"Methapharm, Inc.",6,0.0,0.0
4,Oct 24 2022 12:51PM,249263,10,Methapharm-G,6,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2741707,155,4.0,2.0
15,249241,5,22.0,0.0
18,249252,1,0.0,0.0
19,997000,14,0.0,0.0
20,498507,22,0.0,0.0
21,249234,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2741707,155,4.0,2.0
1,15,249241,5,22.0,0.0
2,18,249252,1,0.0,0.0
3,19,997000,14,0.0,0.0
4,20,498507,22,0.0,0.0
5,21,249234,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,155,4.0,2.0
1,15,5,22.0,0.0
2,18,1,0.0,0.0
3,19,14,0.0,0.0
4,20,22,0.0,0.0
5,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,155.0
1,15,Released,5.0
2,18,Released,1.0
3,19,Released,14.0
4,20,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20,21
Status,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0
Executing,4.0,22.0,0.0,0.0,0.0,0.0
Released,155.0,5.0,1.0,14.0,22.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,22.0,0.0,0.0,0.0,0.0
2,Released,155.0,5.0,1.0,14.0,22.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,22.0,0.0,0.0,0.0,0.0
2,Released,155.0,5.0,1.0,14.0,22.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()